In [20]:
import os
import numpy as np
import pandas as pd
np.random.seed(123)
from six.moves import cPickle

import keras
import tensorflow as tf

from keras import backend as K
# from keras.models import Model
from keras.layers import Input, Dense, Flatten
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.optimizers import Adam

# from prednet import PredNet
# # from data_utils import SequenceGenerator
# from kitti_settings import *

import os
import numpy as np
import PIL
import matplotlib.pyplot as plt
from matplotlib import cm
import time
import multiprocessing
import random
import sklearn
from sklearn import metrics
import seaborn as sns
import cv2
import skimage
from skimage import restoration
import copy

In [2]:
raw_RAD_id_list = os.listdir('/home/hadoop/Documents/Neutrino/SRAD2018/SRAD2018_train/')
print(len(raw_RAD_id_list))
RAD_id_list = raw_RAD_id_list

15000


In [3]:
def check_RAD_id(RAD_id):
#     return RAD_id
    mean_list = []
    for k in range(61):
        mean_list.append(np.array(PIL.Image.open('/home/hadoop/Documents/Neutrino/SRAD2018/SRAD2018_train/%s/%s_%03d.png'
                         % (RAD_id, RAD_id,
                        k))).astype(np.int8).ravel().mean())
    mean_list = np.array(mean_list)
    if mean_list.mean() < -0.5:
        return None
    for k in range(59):
        if abs(mean_list[k] + mean_list[k + 2] - 2 * mean_list[k + 1]) > 2:
            return None
    return RAD_id

In [4]:
pool = multiprocessing.Pool()
start_time = time.time()
print(time.strftime('start time: %Y-%m-%d %H:%M:%S', time.localtime()))
# map(check_RAD_id, raw_RAD_id_list[:100])
# print(time.strftime('start time: %Y-%m-%d %H:%M:%S', time.localtime()))
RAD_id_list = list(pool.map(check_RAD_id, raw_RAD_id_list))
RAD_id_list = [x for x in RAD_id_list if x is not None]
print(time.strftime('end time: %Y-%m-%d %H:%M:%S', time.localtime()))
print(time.strftime('%H:%M:%S', time.gmtime(time.time() - start_time)))
print(len(RAD_id_list))

start time: 2018-09-24 23:01:39
end time: 2018-09-24 23:02:42
00:01:02
6562


Process ForkPoolWorker-32:
Process ForkPoolWorker-29:
Process ForkPoolWorker-31:
Process ForkPoolWorker-24:
Process ForkPoolWorker-14:
Process ForkPoolWorker-5:
Process ForkPoolWorker-15:
Process ForkPoolWorker-18:
Process ForkPoolWorker-21:
Process ForkPoolWorker-13:
Process ForkPoolWorker-27:
Process ForkPoolWorker-8:
Process ForkPoolWorker-25:
Process ForkPoolWorker-20:
Process ForkPoolWorker-6:
Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Process ForkPoolWorker-30:
Process ForkPoolWorker-16:
Process ForkPoolWorker-7:
Process ForkPoolWorker-12:
Process ForkPoolWorker-9:
Process ForkPoolWorker-26:
Process ForkPoolWorker-28:
Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Process ForkPoolWorker-17:
Process ForkPoolWorker-11:
Process ForkPoolWorker-19:
Process ForkPoolWorker-23:
Process ForkPoolWorker-22:
Process ForkPoolWorker-10:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Trace

  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/p

  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
Keyb

In [ ]:
def data_generation(list_IDs_temp, batch_size, image_size, nt, step_size, image_scalar, offset=None, path='/home/hadoop/Documents/Neutrino/SRAD2018/SRAD2018_train'):
    '''
    nt * step_size + offset = 60
    '''
    X = np.empty((batch_size, nt, image_size, image_size, n_channels))
    y = np.empty((batch_size, image_size, image_size, n_channels))
    for i, RAD_id in enumerate(list_IDs_temp):
#         offset = random.randint(0, 61 - nt * step_size)
        if offset == None:
            offset = random.randint(2, 59 - nt * step_size)
        for j in range(nt):
            temp_matrix = np.empty((n_channels, image_size, image_size))
#             temp_matrix[0] = np.array(PIL.Image.open("/home/hadoop/Documents/Neutrino/SRAD2018/SRAD2018_train/%s/%s_%03d.png" % (RAD_id, RAD_id, j * step_size + offset - 1)).resize((image_size, image_size))).astype(np.int8) / image_scalar + \
#                              np.array(PIL.Image.open("/home/hadoop/Documents/Neutrino/SRAD2018/SRAD2018_train/%s/%s_%03d.png" % (RAD_id, RAD_id, j * step_size + offset - 2)).resize((image_size, image_size))).astype(np.int8) / image_scalar
            temp_matrix[0] = np.array(PIL.Image.open("%s/%s/%s_%03d.png" % (path, RAD_id, RAD_id, j * step_size + offset)).resize((image_size, image_size))).astype(np.int8) / image_scalar
#             temp_matrix[2] = np.array(PIL.Image.open("/home/hadoop/Documents/Neutrino/SRAD2018/SRAD2018_train/%s/%s_%03d.png" % (RAD_id, RAD_id, j * step_size + offset + 1)).resize((image_size, image_size))).astype(np.int8) / image_scalar + \
#                              np.array(PIL.Image.open("/home/hadoop/Documents/Neutrino/SRAD2018/SRAD2018_train/%s/%s_%03d.png" % (RAD_id, RAD_id, j * step_size + offset + 2)).resize((image_size, image_size))).astype(np.int8) / image_scalar
            temp_matrix[1] = cv2.GaussianBlur(temp_matrix[0], (5, 5), 0)
            temp_matrix[2] = cv2.GaussianBlur(temp_matrix[0], (9, 9), 0)
            temp_matrix[3] = cv2.GaussianBlur(temp_matrix[0], (13, 13), 0)
            temp_matrix = np.rollaxis(temp_matrix, 0, 3)
            X[i][j] = temp_matrix
        temp_matrix = np.empty((n_channels, image_size, image_size))
        temp_matrix[0] = np.array(PIL.Image.open("%s/%s/%s_%03d.png" % (path, RAD_id, RAD_id, nt * step_size + offset)).resize((image_size, image_size))).astype(np.int8) / image_scalar
        temp_matrix[1] = cv2.GaussianBlur(temp_matrix[0], (5, 5), 0)
        temp_matrix[2] = cv2.GaussianBlur(temp_matrix[0], (9, 9), 0)
        temp_matrix[3] = cv2.GaussianBlur(temp_matrix[0], (13, 13), 0)
        temp_matrix = np.rollaxis(temp_matrix, 0, 3)
        y[i] = temp_matrix
    return X, y

In [5]:
30 // 30

1

In [12]:
a = np.array(PIL.Image.open("/home/hadoop/Documents/Neutrino/SRAD2018/SRAD2018_train/%s/%s_%03d.png" % (RAD_id, RAD_id, 30)).resize((image_size, image_size))).astype(np.int8).reshape((image_size, image_size))

In [53]:
from sklearn import linear_model
reg = sklearn.linear_model.LinearRegression()
reg.fit([[0], [1], [2]], [0, 3, 6])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [54]:
reg.predict([[10]])

array([30.])

In [88]:
def model_predict(x):
    return cv2.GaussianBlur(x[-1], (5, 5), 0)
    ans = np.empty((image_size, image_size))
    for i in range(image_size):
        for j in range(image_size):
#             reg_x = []
#             reg_y = []
#             for k in range(nt):
#                 reg_x.append([k])
#                 reg_y.append(x[k][i][j])
#             reg = sklearn.linear_model.LinearRegression()
#             reg.fit(reg_x, reg_y)
            k = (x[-1][i][j] - x[0][i][j]) / (nt - 1)
            m = x[-1][i][j]
            ans[i][j] = m + k
    return ans

In [93]:
image_size = 512
nt = 3
kernel_size = 15

In [94]:
start_time = time.time()
print(time.strftime("start time: %Y-%m-%d %H:%M:%S", time.localtime()))
model_abs_loss = []
model_sqr_loss = []
last_frame_abs_loss = []
last_frame_sqr_loss = []
for RAD_id_counter, RAD_id in enumerate(RAD_id_list[-50:]):
    x = np.empty((nt, image_size, image_size))
    for i in range(nt):
        x[i] = np.array(PIL.Image.open("/home/hadoop/Documents/Neutrino/SRAD2018/SRAD2018_train/%s/%s_%03d.png" % (RAD_id, RAD_id, 31 + i - nt)).resize((image_size, image_size))).astype(np.int8).reshape((image_size, image_size))
    x_last = np.array(PIL.Image.open("/home/hadoop/Documents/Neutrino/SRAD2018/SRAD2018_train/%s/%s_%03d.png" % (RAD_id, RAD_id, 30)).resize((image_size, image_size))).astype(np.int8).reshape((image_size, image_size))
    y_true_last = np.array(PIL.Image.open("/home/hadoop/Documents/Neutrino/SRAD2018/SRAD2018_train/%s/%s_%03d.png" % (RAD_id, RAD_id, 60)).resize((image_size, image_size))).astype(np.int8).reshape((image_size, image_size))
    for i in range(30 // 30):  # 30 // nt
#         y = model_predict(x)
        y = cv2.GaussianBlur(x[-1], (kernel_size, kernel_size), 0)
        for j in range(nt - 1):
            x[j] = x[j + 1]
        x[-1] = y
    y_pred_last = y
    last_frame_abs_loss.append(sklearn.metrics.mean_absolute_error(y_true_last, x_last))
    model_abs_loss.append(sklearn.metrics.mean_absolute_error(y_true_last, y_pred_last))
    last_frame_sqr_loss.append(sklearn.metrics.mean_squared_error(y_true_last.astype(np.int), x_last.astype(np.int)))
    model_sqr_loss.append(sklearn.metrics.mean_squared_error(y_true_last.astype(np.int), y_pred_last.astype(np.int)))
    if RAD_id_counter % 10 == 0:
        print('RAD_id_counter=%5d\t\tIt takes %.2f' % (RAD_id_counter, time.time() - start_time))
model_abs_loss = np.array(model_abs_loss)
last_frame_abs_loss = np.array(last_frame_abs_loss)
model_sqr_loss = np.array(model_sqr_loss)
last_frame_sqr_loss = np.array(last_frame_sqr_loss)
print('     model_abs_loss=%.7f' % model_abs_loss.mean())
print('last_frame_abs_loss=%.7f' % last_frame_abs_loss.mean())
print('     model_sqr_loss=%.7f' % model_sqr_loss.mean())
print('last_frame_sqr_loss=%.7f' % last_frame_sqr_loss.mean())
print(time.strftime('It took  %H:%M:%S', time.gmtime(time.time() - start_time)))
print(time.strftime("end time: %Y-%m-%d %H:%M:%S", time.localtime()))

start time: 2018-09-25 00:33:26
RAD_id_counter=    0		It takes 0.01
RAD_id_counter=   10		It takes 0.14
RAD_id_counter=   20		It takes 0.25
RAD_id_counter=   30		It takes 0.37
RAD_id_counter=   40		It takes 0.48
     model_abs_loss=2.9634400
last_frame_abs_loss=3.0182784
     model_sqr_loss=55.0416141
last_frame_sqr_loss=62.5443438
It took  00:00:00
end time: 2018-09-25 00:33:26


In [109]:
sigmaX = 100
kernel_size = 51
for kernel_size in range(3, 81, 6):
    start_time = time.time()
    print('kernel_size=', kernel_size)
    print(time.strftime("start time: %Y-%m-%d %H:%M:%S", time.localtime()))
    model_abs_loss = []
    model_sqr_loss = []
    last_frame_abs_loss = []
    last_frame_sqr_loss = []
    for RAD_id_counter, RAD_id in enumerate(RAD_id_list[-50:]):
        x = np.empty((nt, image_size, image_size))
        for i in range(nt):
            x[i] = np.array(PIL.Image.open("/home/hadoop/Documents/Neutrino/SRAD2018/SRAD2018_train/%s/%s_%03d.png" % (RAD_id, RAD_id, 31 + i - nt)).resize((image_size, image_size))).astype(np.int8).reshape((image_size, image_size))
        x_last = np.array(PIL.Image.open("/home/hadoop/Documents/Neutrino/SRAD2018/SRAD2018_train/%s/%s_%03d.png" % (RAD_id, RAD_id, 30)).resize((image_size, image_size))).astype(np.int8).reshape((image_size, image_size))
        y_true_last = np.array(PIL.Image.open("/home/hadoop/Documents/Neutrino/SRAD2018/SRAD2018_train/%s/%s_%03d.png" % (RAD_id, RAD_id, 60)).resize((image_size, image_size))).astype(np.int8).reshape((image_size, image_size))
        for i in range(30 // 30):  # 30 // nt
    #         y = model_predict(x)
            y = cv2.GaussianBlur(x[-1], (kernel_size, kernel_size), sigmaX)
            for j in range(nt - 1):
                x[j] = x[j + 1]
            x[-1] = y
        y_pred_last = y
        last_frame_abs_loss.append(sklearn.metrics.mean_absolute_error(y_true_last.astype(np.int), x_last.astype(np.int)))
        model_abs_loss.append(sklearn.metrics.mean_absolute_error(y_true_last.astype(np.int), y_pred_last.astype(np.int)))
        last_frame_sqr_loss.append(sklearn.metrics.mean_squared_error(y_true_last.astype(np.int), x_last.astype(np.int)))
        model_sqr_loss.append(sklearn.metrics.mean_squared_error(y_true_last.astype(np.int), y_pred_last.astype(np.int)))
#         if RAD_id_counter % 10 == 0:
#             print('RAD_id_counter=%5d\t\tIt takes %.2f' % (RAD_id_counter, time.time() - start_time))
    model_abs_loss = np.array(model_abs_loss)
    last_frame_abs_loss = np.array(last_frame_abs_loss)
    model_sqr_loss = np.array(model_sqr_loss)
    last_frame_sqr_loss = np.array(last_frame_sqr_loss)
    print('     model_abs_loss=%.7f' % model_abs_loss.mean())
    print('last_frame_abs_loss=%.7f' % last_frame_abs_loss.mean())
    print('     model_sqr_loss=%.7f' % model_sqr_loss.mean())
    print('last_frame_sqr_loss=%.7f' % last_frame_sqr_loss.mean())
    print(time.strftime('It took  %H:%M:%S', time.gmtime(time.time() - start_time)))
    print(time.strftime("end time: %Y-%m-%d %H:%M:%S", time.localtime()))
    print('kernel_size=', kernel_size)

kernel_size= 3
start time: 2018-09-25 01:05:21
     model_abs_loss=2.9467916
last_frame_abs_loss=3.0182784
     model_sqr_loss=58.8740609
last_frame_sqr_loss=62.5443438
It took  00:00:00
end time: 2018-09-25 01:05:22
kernel_size= 3
kernel_size= 9
start time: 2018-09-25 01:05:22
     model_abs_loss=2.9312921
last_frame_abs_loss=3.0182784
     model_sqr_loss=55.1349142
last_frame_sqr_loss=62.5443438
It took  00:00:00
end time: 2018-09-25 01:05:22
kernel_size= 9
kernel_size= 15
start time: 2018-09-25 01:05:22
     model_abs_loss=2.9270804
last_frame_abs_loss=3.0182784
     model_sqr_loss=52.4270488
last_frame_sqr_loss=62.5443438
It took  00:00:00
end time: 2018-09-25 01:05:23
kernel_size= 15
kernel_size= 21
start time: 2018-09-25 01:05:23
     model_abs_loss=2.9278022
last_frame_abs_loss=3.0182784
     model_sqr_loss=50.3546018
last_frame_sqr_loss=62.5443438
It took  00:00:00
end time: 2018-09-25 01:05:24
kernel_size= 21
kernel_size= 27
start time: 2018-09-25 01:05:24
     model_abs_loss=

In [31]:
y_pred_last.astype(np.int8)

array([[-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]], dtype=int8)

In [33]:
sklearn.metrics.mean_squared_error(x_last, y_true_last.astype(np.int))

92.82624435424805

In [28]:
sklearn.metrics.mean_squared_error(y_pred_last, y_true_last)

92.82624435424805

In [32]:
sklearn.metrics.mean_squared_error(y_true_last, y_pred_last.astype(np.int8))

-0.3251228332519531

In [ ]:
model_abs_loss = np.array(model_abs_loss)
last_frame_abs_loss = np.array(last_frame_abs_loss)
model_sqr_loss = np.array(model_sqr_loss)
last_frame_sqr_loss = np.array(last_frame_sqr_loss)
print('     model_abs_loss=%.7f' % model_abs_loss.mean())
print('last_frame_abs_loss=%.7f' % last_frame_abs_loss.mean())
print('     model_sqr_loss=%.7f' % model_sqr_loss.mean())
print('last_frame_sqr_loss=%.7f' % last_frame_sqr_loss.mean())

sqr:

15.3290176 0
15.3366709 1
15.2775345 2
15.2723169 average

27.9708243 0
24.8304856 0.1
21.4680993 0.2
21.1976629 0.25
21.4486480 0.3
23.0941604 0.4

abs:

1.4878454 average

1.4255825 0.25

In [ ]:
y.shape

In [ ]:
x_last = x_last.reshape(image_size, image_size)
plt.imshow(x_last, cmap=cm.gist_ncar_r, vmin=vmin, vmax=vmax)
plt.show()
y_true_last = y_true_last.reshape(image_size, image_size)
plt.imshow(y_true_last, cmap=cm.gist_ncar_r, vmin=vmin, vmax=vmax)
plt.show()
y_pred_last = y_pred_last.reshape(image_size, image_size)
plt.imshow(y_pred_last, cmap=cm.gist_ncar_r, vmin=vmin, vmax=vmax)
plt.show()
plt.imshow(rescale(x_last, y_pred_last), cmap=cm.gist_ncar_r, vmin=vmin, vmax=vmax)
plt.show()

# 用模型预测数据

In [107]:
RAD_id_list = os.listdir("D:\SRAD2018\SRAD2018_Test_1")
print(len(RAD_id_list))

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\SRAD2018\\SRAD2018_Test_1'